In [ ]:
!pip install pyspellchecker

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
import spacy
import string
from spellchecker import SpellChecker
pd.options.mode.chained_assignment = None

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
x_train = pd.read_csv(r'https://raw.githubusercontent.com/Ignacio-Ibarra/NLP-Disasters/intento_3_nachos/intento_3/train_clean.csv',usecols=['id','text'])
y_train = pd.read_csv(r'https://raw.githubusercontent.com/Ignacio-Ibarra/NLP-Disasters/intento_3_nachos/intento_3/train_clean.csv',usecols= ['target'])
x_test = pd.read_csv(r'https://raw.githubusercontent.com/Ignacio-Ibarra/NLP-Disasters/intento_3_nachos/intento_3/test_clean.csv', usecols = ['id','text'])

In [ ]:
x_train['istrain'] = 1
x_test['istrain'] = 0
complete = x_train.append(x_test)

In [ ]:
complete['text'] = complete["text"].astype(str)

In [ ]:
complete["text_lower"] = complete["text"].str.lower()

In [ ]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

complete["text_remove_punct"] = complete["text_lower"].apply(lambda text: remove_punctuation(text))

In [ ]:
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

complete["text_wo_stop"] = complete["text_remove_punct"].apply(lambda text: remove_stopwords(text))

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

complete["text_wo_emojis"] = complete["text_wo_stop"].apply(lambda text: remove_emoji(text))

In [ ]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

complete["text_wo_html"] = complete["text_wo_emojis"].apply(lambda text: remove_html(text))

In [77]:
#Removing chat words

#This is a pandas df with many types of "chat words"
slang = pd.read_csv(r'/content/slang.txt', sep = '=', names = ['slang','meaning'])

def replace_chat_word(text): 
  line = []
  for i in text.split(): 
    if i in slang.slang.to_list(): 
      i = slang.loc[slang.slang == i,'meaning'].values[0].lower()
      line.append(i)
    else: 
      line.append(i)
  return str(" ".join(line))

complete['text_no_chat_word'] = complete["text_wo_html"].apply(lambda x: replace_chat_word(x))

In [78]:
import emot
from emot.core import emoji
from emot.core import emoticons
from emot.emo_unicode import EMO_UNICODE
from emot.emo_unicode import UNICODE_EMO
from emot.emo_unicode import EMOTICONS

In [81]:
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

complete['text_clean']=complete['text_no_chat_word'].apply(lambda x: remove_emoticons(x))

In [83]:
complete = complete[['id', 'text','text_clean', 'istrain']]
x_train = complete.loc[complete.istrain == 1,:]
x_test = complete.loc[complete.istrain == 0,:]

In [89]:
x_train.to_csv('train_clean.csv', index=False)
x_test.to_csv('test_clean.csv', index=False)
y_train.to_csv('targets.csv', index=False)